In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from pathlib import Path
from zipfile import ZipFile
import pandas as pd
import torch
import random
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm, trange

import sys
sys.path.append('..')

from src.DMHP.HP import PointProcessStorage, DirichletMixtureModel, EM_clustering, tune_basis_fn
from src.DMHP.metrics import consistency, purity
from src.Cohortney.data_utils import load_data

## Real data

### IPTV

In [ ]:
path = Path('data', 'IPTV_Data')
ss, Ts, class2idx, _ = load_data(path, ext='txt', datetime=True)

N = len(ss)
# D = 5
# basis_fs = [lambda x: torch.exp(- x**2 / (2.*(k+1)**2) ) for k in range(D)]

eps = 1e6
basis_fs = tune_basis_fn(ss, eps=eps)
D = len(basis_fs)    
    
C = len(class2idx)
K = 6

ntrials = 5
niter = 10

labels = torch.zeros(ntrials, len(ss))
nlls = torch.zeros(ntrials, niter)

for i in trange(ntrials):
    Sigma = torch.rand(C, C, D, K)
    B = torch.rand(C, K)
    alpha = 1.

    train_ids = np.sort(np.random.choice(np.arange(len(ss)), size=len(ss) // 2, replace=False))
    train_fold = [ss[i] for i in range(len(ss)) if i in train_ids]
    train_Ts = Ts[train_ids]
    
    # learn
    hp = PointProcessStorage(train_fold, train_Ts, basis_fs)
    model = DirichletMixtureModel(K, C, D, alpha, B, Sigma)
    EM = EM_clustering(hp, model, n_inner=5)

    _, nll_history, _ = EM.learn_hp(niter=niter, ninner=[2,3,4,5,6,7] + (niter-6)*[8])

    # validate
    EM.hp = PointProcessStorage(ss, Ts, basis_fs)
    EM.N = len(ss)
    EM.int_g = []
    EM.g = []
    r = EM.e_step()
    
    labels[i] = r.argmax(-1)
    nlls[i] = torch.FloatTensor(nll_history)

    print(f'\nConsistency of clustering: {consistency(labels[:i+1]).item():.4f}')
    
print(f'Consistency of clustering: {consistency(labels).item():.4f}')

  0%|          | 0/5 [00:00<?, ?it/s]
0it [00:00, ?it/s]

0.15015015015015015
D = 17


In [6]:
# new basis
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/IPTV_Data' \
    --verbose \
    --nruns 5 \
    --ext txt \
    --nmb_cluster 6

0.15015015015015015
D = 17
============= RUN 1 ===============
0it [00:00, ?it/s]Killed


In [22]:
# plt.figure(figsize=(9, 5))
# plt.grid()
# plt.plot(np.arange(niter)+1, nlls.mean(0).numpy() / len(train_ids))
# plt.fill_between(np.arange(niter)+1, (nlls.mean(0).numpy() - nlls.std(0).numpy()) / len(train_ids), (nlls.mean(0).numpy() + nlls.std(0).numpy()) / len(train_ids), alpha=0.2)
# plt.title('Mixing DMMHP', fontsize=15)
# plt.xlabel(r'$n$ outer iterations', fontsize=15)
# plt.ylabel(r'$\sim$ NLL / $N$', fontsize=15)
# plt.show()

### Booking

In [33]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/booking_tmp_tmp' \
    --verbose \
    --nruns 5 \
    --ext csv \
    --nmb_cluster 6 \
    --type 'booking2'

0.06006006006006006
D = 7
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 168.0827
1it [00:09,  9.75s/it]
NLL / N: 129.0512
2it [00:19,  9.89s/it]
NLL / N: 130.2115
3it [00:33, 10.94s/it]
NLL / N: 129.3995
4it [00:52, 13.37s/it]
NLL / N: 129.4784
5it [01:13, 15.61s/it]
NLL / N: 129.9099
6it [01:38, 18.45s/it]
NLL / N: 131.1426
7it [02:06, 21.38s/it]
NLL / N: 132.2076
8it [02:35, 23.57s/it]
NLL / N: 133.3726
9it [03:03, 25.01s/it]
NLL / N: 134.5327
10it [03:31, 21.18s/it]
preds: tensor([1., 0., 5., 0., 1., 1., 4., 4., 4., 3., 3., 1., 5., 4., 1., 1., 5., 4.,
        0., 0., 3., 4., 0., 4., 1., 4., 3., 3., 1., 1., 2., 4., 1., 4., 0., 1.,
        1., 1., 0., 1., 1., 5., 4., 1., 1., 1., 1., 0., 1.])
============= RUN 2 ===============
0it [00:00, ?it/s]
NLL / N: 150.2061
1it [00:08,  8.24s/it]
NLL / N: 129.8977
2it [00:18,  9.00s/it]
NLL / N: 128.9243
3it [00:34, 10.84s/it]
NLL / N: 130.0245
4it [00:52, 13.06s/it]
NLL / N: 129.9674
5it [01:14, 15.73s/it]
NLL / N: 130.3037
6i

## Simulated

### Adaptive basis

In [24]:
# new basis
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K3_C1' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 3

0.22522522522522523
D = 15
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 88.2518
1it [12:13, 733.50s/it]
NLL / N: -33.9981
2it [14:03, 546.47s/it]
NLL / N: -18.2411
3it [16:22, 424.19s/it]
NLL / N: -8.6365
4it [19:10, 347.24s/it]
NLL / N: -2.8665
5it [22:28, 302.58s/it]
NLL / N: 0.0569
6it [26:12, 278.99s/it]
NLL / N: 2.4243
7it [30:27, 271.76s/it]
NLL / N: 3.2875
8it [34:40, 266.20s/it]
NLL / N: 3.8015
9it [38:55, 262.69s/it]
NLL / N: 3.5913
10it [43:05, 258.50s/it]
preds: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.,

Consistency: 0.6907

Purity: 0.7753+-0.0980


In [9]:
# new basis
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K2_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 2

0.24024024024024024
D = 6
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 100.3893
1it [02:16, 136.60s/it]
NLL / N: 112.4715
2it [02:22, 97.38s/it] 
NLL / N: 93.3330
3it [02:30, 70.49s/it]
NLL / N: 99.9873
4it [02:38, 51.94s/it]
NLL / N: 102.8767
5it [02:49, 39.65s/it]
NLL / N: 99.1331
6it [03:01, 31.39s/it]
NLL / N: 96.1592
7it [03:15, 25.98s/it]
NLL / N: 95.1458
8it [03:29, 22.44s/it]
NLL / N: 95.1242
9it [03:44, 20.11s/it]
NLL / N: 94.9622
10it [04:01, 24.13s/it]
preds: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1.

4it [02:47, 55.08s/it]
NLL / N: 84.7142
5it [02:57, 41.39s/it]
NLL / N: 89.9297
6it [03:08, 32.28s/it]
NLL / N: 101.2661
7it [03:20, 26.28s/it]
NLL / N: 100.1482
8it [03:33, 22.20s/it]
NLL / N: 97.4298
9it [03:47, 19.94s/it]
NLL / N: 95.5454
10it [04:05, 24.50s/it]
preds: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       

In [10]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K3_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 3

0.46546546546546547
D = 15
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 113.7754
1it [08:43, 523.11s/it]
NLL / N: 113.7483
2it [09:13, 375.25s/it]
NLL / N: 121.2475
3it [11:16, 299.66s/it]
NLL / N: 126.8787
4it [13:09, 243.60s/it]
NLL / N: 114.4070
5it [14:29, 194.66s/it]
NLL / N: 116.9374
6it [14:57, 144.44s/it]
NLL / N: 115.5947
7it [16:26, 127.85s/it]
NLL / N: 114.0703
8it [20:23, 160.59s/it]
NLL / N: 113.1959
9it [21:37, 134.83s/it]
NLL / N: 112.8448
10it [22:35, 135.50s/it]
preds: tensor([1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
        1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
        0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
        1.

0it [00:00, ?it/s]
NLL / N: 108.5582
1it [09:08, 548.36s/it]
NLL / N: 117.8121
2it [09:33, 391.26s/it]
NLL / N: 116.3263
3it [09:50, 279.07s/it]
NLL / N: 115.3251
4it [10:19, 204.06s/it]
NLL / N: 119.2426
5it [12:36, 184.09s/it]
NLL / N: 117.1642
6it [14:46, 167.68s/it]
NLL / N: 114.0231
7it [16:47, 153.68s/it]
NLL / N: 112.8355
8it [17:17, 116.74s/it]
NLL / N: 112.5074
9it [18:51, 109.80s/it]
NLL / N: 112.3795
10it [20:52, 125.25s/it]
preds: tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
        1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
        0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0.,
        1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

0it [00:00, ?it/s]
NLL / N: 107.9426
1it [09:08, 548.52s/it]
NLL / N: 116.4548
2it [09:22, 388.23s/it]
NLL / N: 120.6777
3it [09:40, 277.02s/it]
NLL / N: 121.5890
4it [10:58, 217.53s/it]
NLL / N: 116.4306
5it [12:06, 172.67s/it]
NLL / N: 112.1409
6it [13:58, 154.24s/it]
NLL / N: 111.2436
7it [15:58, 144.13s/it]
NLL / N: 111.0736
8it [16:30, 110.33s/it]
NLL / N: 111.0714
9it [18:31, 113.54s/it]
NLL / N: 111.0606
10it [20:13, 121.36s/it]
preds: tensor([2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 0., 2., 2., 0., 2., 2., 2., 2., 2., 2., 2., 0., 2., 2., 2., 2., 2.,
        2., 2., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0.,
        2., 0., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        0., 2., 2., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2

In [11]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K4_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 4

/mnt/Cohortney/src/mds20_cohortney-main/src/DMHP/HP.py:26: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  result = lambda w0: integrate.quad(upper_bound, w0, np.inf)
0.7207207207207207
D = 20
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 105.2193
1it [14:31, 871.02s/it]
NLL / N: 110.9393
2it [14:50, 615.46s/it]
NLL / N: 115.1564
3it [16:34, 461.97s/it]
NLL / N: 115.5475
4it [17:12, 335.05s/it]
NLL / N: 115.1132
5it [18:05, 250.23s/it]
NLL / N: 114.8754
6it [20:33, 219.77s/it]
NLL / N: 114.7175
7it [22:17, 184.77s/it]
NLL / N: 114.6050
8it [24:22, 167.12s/it]
NLL / N: 114.5136
9it [26:34, 156.50s/it]
NLL / N: 114.4491
10it [29:50, 179.08s/it]
preds: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 

6it [23:12, 238.15s/it]
NLL / N: 113.8491
7it [26:30, 226.24s/it]
NLL / N: 113.8433
8it [28:08, 187.52s/it]
NLL / N: 113.8085
9it [29:47, 161.04s/it]
NLL / N: 113.6501
10it [30:55, 185.53s/it]
preds: tensor([3., 3., 2., 3., 3., 3., 3., 2., 2., 2., 2., 3., 2., 3., 3., 2., 3., 2.,
        2., 3., 3., 2., 3., 3., 2., 2., 2., 2., 2., 3., 3., 3., 2., 3., 3., 2.,
        2., 2., 2., 2., 2., 3., 2., 2., 2., 2., 3., 2., 2., 3., 2., 2., 2., 3.,
        2., 2., 2., 2., 3., 2., 2., 2., 3., 3., 2., 3., 2., 2., 2., 2., 2., 3.,
        3., 2., 2., 2., 2., 2., 2., 3., 2., 2., 3., 3., 2., 2., 2., 3., 2., 3.,
        2., 2., 3., 2., 3., 3., 3., 2., 2., 2., 2., 3., 3., 2., 3., 2., 3., 3.,
        2., 2., 2., 2., 2., 3., 2., 3., 3., 2., 2., 3., 1., 3., 2., 2., 2., 2.,
        2., 2., 2., 2., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3.,
        2., 3., 2., 3., 3., 2., 3., 2., 2., 2., 3., 3., 3., 3., 2., 2., 2., 3.,
        3., 2., 3., 2., 2., 3., 2., 2., 2., 2., 2., 3., 3., 2., 3., 2., 2., 2.,


0it [00:00, ?it/s]
NLL / N: 108.6018
1it [15:04, 904.53s/it]
NLL / N: 113.7024
2it [15:57, 649.06s/it]
NLL / N: 114.3328
3it [16:43, 468.15s/it]
NLL / N: 111.9531
4it [17:20, 338.72s/it]
NLL / N: 111.9587
5it [18:21, 255.40s/it]
NLL / N: 113.0914
6it [19:25, 198.14s/it]
NLL / N: 114.1746
7it [20:36, 159.89s/it]
NLL / N: 113.5475
8it [21:52, 134.81s/it]
NLL / N: 113.0695
9it [22:55, 113.35s/it]
NLL / N: 113.0943
10it [24:08, 144.83s/it]
preds: tensor([3., 3., 3., 3., 3., 0., 3., 3., 3., 3., 3., 3., 3., 0., 0., 3., 0., 3.,
        3., 3., 0., 3., 3., 3., 3., 3., 3., 3., 3., 3., 0., 0., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 0., 3., 3., 3., 3., 3., 3., 3., 0., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 0.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 0., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 0., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 0., 3., 3., 3., 3., 1., 3., 3., 3., 3

In [12]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K5_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 5

0.9009009009009009
D = 29
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 103.3651
1it [26:49, 1609.37s/it]
NLL / N: 112.9595
2it [27:49, 1144.60s/it]
NLL / N: 112.6651
3it [28:25, 812.01s/it] 
NLL / N: 114.2843
4it [29:24, 586.23s/it]
NLL / N: 125.7112
5it [31:06, 440.91s/it]
NLL / N: 133.3572
6it [33:08, 345.06s/it]
NLL / N: 145.4225
7it [35:41, 287.63s/it]
NLL / N: 151.7624
8it [38:04, 244.07s/it]
NLL / N: 152.7385
9it [39:29, 196.51s/it]
NLL / N: 150.1396
10it [42:07, 252.74s/it]
preds: tensor([3., 3., 0., 3., 0., 0., 0., 0., 3., 0., 0., 3., 3., 3., 0., 3., 0., 0.,
        3., 3., 3., 0., 3., 0., 0., 0., 3., 0., 0., 3., 3., 0., 0., 0., 1., 0.,
        3., 0., 3., 3., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 3.,
        3., 3., 0., 3., 3., 3., 0., 0., 0., 0., 0., 0., 0., 3., 0., 3., 1., 0.,
        0., 3., 0., 0., 0., 0., 3., 0., 0., 0., 3., 3., 0., 3., 3., 3., 0., 3.,
        3., 0., 0., 0., 0., 3., 3., 3., 0., 0., 3., 3., 3., 3., 3., 3., 3., 0.,
        

0it [00:00, ?it/s]
NLL / N: 103.6645
1it [32:41, 1961.50s/it]
NLL / N: 105.1671
2it [33:37, 1389.78s/it]
NLL / N: 109.1184
3it [35:15, 1002.42s/it]
NLL / N: 108.2205
4it [36:22, 721.57s/it] 
NLL / N: 108.0639
5it [38:35, 545.12s/it]
NLL / N: 109.2735
6it [41:02, 425.64s/it]
NLL / N: 109.9033
7it [43:37, 344.53s/it]
NLL / N: 110.2708
8it [45:15, 270.44s/it]
NLL / N: 110.6243
9it [47:50, 235.76s/it]
NLL / N: 110.7988
10it [50:38, 303.84s/it]
preds: tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 0., 3.,
        3., 3., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1.,
        3., 1., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 1., 3., 3., 0., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1

0it [00:00, ?it/s]
NLL / N: 106.3270
1it [31:22, 1882.36s/it]
NLL / N: 106.6308
2it [32:15, 1333.58s/it]
NLL / N: 106.1097
3it [33:51, 962.22s/it] 
NLL / N: 106.9749
4it [34:41, 688.68s/it]
NLL / N: 113.7982
5it [37:07, 525.83s/it]
NLL / N: 114.0345
6it [39:16, 406.73s/it]
NLL / N: 113.4614
7it [42:04, 335.21s/it]
NLL / N: 113.2726
8it [43:21, 257.58s/it]
NLL / N: 113.0521
9it [45:49, 224.81s/it]
NLL / N: 112.8403
10it [48:22, 290.29s/it]
preds: tensor([3., 2., 0., 3., 0., 3., 3., 4., 3., 4., 3., 3., 3., 3., 4., 3., 4., 4.,
        3., 3., 3., 3., 3., 3., 3., 4., 3., 3., 4., 3., 3., 3., 4., 3., 0., 4.,
        3., 3., 2., 2., 3., 3., 3., 3., 3., 3., 4., 4., 3., 4., 4., 3., 0., 2.,
        3., 2., 4., 3., 3., 3., 4., 3., 3., 3., 4., 4., 3., 3., 4., 3., 0., 3.,
        3., 3., 4., 3., 3., 3., 3., 0., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 4., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4.,
        3., 3., 4., 3., 2., 3., 3., 3., 4., 3., 3., 3., 3., 4., 4., 2.

0it [00:00, ?it/s]
NLL / N: 107.4436
1it [33:07, 1987.95s/it]
NLL / N: 108.5427
2it [33:48, 1403.63s/it]
NLL / N: 111.4757
3it [35:03, 1005.29s/it]
NLL / N: 113.2909
4it [35:46, 716.50s/it] 
NLL / N: 114.1223
5it [37:50, 538.79s/it]
NLL / N: 114.0531
6it [39:16, 402.93s/it]
NLL / N: 114.6111
7it [40:49, 309.79s/it]
NLL / N: 116.5037
8it [42:57, 255.45s/it]
NLL / N: 140.8665
9it [43:55, 195.98s/it]
NLL / N: 194.4540
10it [45:52, 275.24s/it]
preds: tensor([1., 1., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 1., 1., 1., 1., 1., 1., 3., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 1.,
        1., 1., 1., 1., 1., 3., 1., 4., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3

0it [00:00, ?it/s]
NLL / N: 105.2015
1it [34:19, 2059.40s/it]
NLL / N: 105.4832
2it [34:40, 1447.92s/it]
NLL / N: 110.5389
3it [35:33, 1029.55s/it]
NLL / N: 109.3102
4it [37:27, 754.69s/it] 
NLL / N: 107.5144
5it [38:24, 545.35s/it]
NLL / N: 108.4651
6it [40:34, 420.73s/it]
NLL / N: 106.8451
7it [42:48, 334.89s/it]
NLL / N: 108.4109
8it [45:00, 273.84s/it]
NLL / N: 110.8672
9it [45:53, 207.66s/it]
NLL / N: 112.0314
10it [48:37, 291.74s/it]
preds: tensor([3., 4., 4., 4., 4., 4., 4., 4., 4., 2., 4., 4., 4., 4., 4., 4., 4., 4.,
        0., 0., 4., 4., 3., 4., 4., 4., 4., 4., 4., 0., 3., 4., 4., 4., 2., 2.,
        4., 4., 3., 4., 4., 4., 4., 4., 4., 4., 4., 3., 4., 4., 4., 4., 2., 0.,
        4., 3., 4., 4., 0., 3., 2., 4., 4., 4., 4., 4., 4., 4., 4., 3., 4., 4.,
        4., 0., 4., 4., 4., 4., 4., 2., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4., 4., 4., 0., 0., 4., 4., 4., 4., 0., 4., 4., 0., 0., 2.,
        4., 4., 4., 0., 3., 4., 3., 4., 4., 4., 4., 4., 4., 4., 2., 3

Purity: 0.6131+-0.0390


In [13]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_sin_K2_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 2

2.0720720720720722
D = 7
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 55.0706
1it [03:36, 216.24s/it]
NLL / N: 42.3252
2it [03:48, 155.13s/it]
NLL / N: 40.7978
3it [04:04, 113.22s/it]
NLL / N: 39.7464
4it [04:23, 85.06s/it] 
NLL / N: 39.9118
5it [04:54, 68.93s/it]
NLL / N: 40.5480
6it [06:35, 78.49s/it]
NLL / N: 41.1702
7it [07:04, 63.51s/it]
NLL / N: 41.6245
8it [07:40, 55.29s/it]
NLL / N: 41.9515
9it [08:26, 52.62s/it]
NLL / N: 42.1856
10it [09:44, 58.49s/it]
preds: tensor([0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
        1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0.,
        1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 0., 0., 

4it [04:55, 93.35s/it]
NLL / N: 39.9840
5it [05:25, 74.30s/it]
NLL / N: 40.6842
6it [06:03, 63.60s/it]
NLL / N: 41.5380
7it [06:32, 53.09s/it]
NLL / N: 41.9635
8it [07:05, 47.09s/it]
NLL / N: 42.2122
9it [08:06, 51.37s/it]
NLL / N: 42.3579
10it [09:31, 57.15s/it]
preds: tensor([1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0.,
        1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
        0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
        0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1.,
        0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
        1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
        0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
        0

In [14]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_sin_K3_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 3

2.6126126126126126
D = 9
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 57.2194
1it [05:18, 318.08s/it]
NLL / N: 46.2563
2it [06:46, 249.21s/it]
NLL / N: 44.2651
3it [07:21, 184.86s/it]
NLL / N: 43.5492
4it [07:48, 137.45s/it]
NLL / N: 43.5737
5it [08:35, 110.36s/it]
NLL / N: 43.6047
6it [10:36, 113.69s/it]
NLL / N: 44.5779
7it [11:19, 92.28s/it] 
NLL / N: 46.6789
8it [12:08, 79.40s/it]
NLL / N: 47.6991
9it [14:05, 90.65s/it]
NLL / N: 48.0550
10it [14:44, 88.50s/it]
preds: tensor([2., 0., 2., 2., 2., 2., 2., 0., 0., 2., 0., 2., 0., 2., 0., 2., 2., 2.,
        1., 0., 1., 2., 0., 2., 2., 2., 0., 2., 2., 2., 0., 2., 2., 0., 2., 2.,
        0., 2., 1., 0., 2., 2., 0., 2., 2., 0., 1., 2., 0., 2., 0., 2., 2., 2.,
        0., 2., 2., 0., 2., 2., 0., 0., 2., 2., 0., 0., 2., 0., 0., 2., 0., 2.,
        2., 0., 2., 0., 2., 0., 2., 2., 2., 0., 2., 2., 2., 1., 0., 0., 0., 0.,
        0., 0., 2., 2., 1., 1., 0., 1., 0., 2., 2., 1., 2., 1., 0., 2., 2., 0.,
        0., 0., 0., 1., 0

In [15]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_sin_K4_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 4

3.813813813813814
D = 13
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 49.4565
1it [12:09, 729.79s/it]
NLL / N: 43.8202
2it [13:37, 537.10s/it]
NLL / N: 42.4807
3it [14:26, 390.81s/it]
NLL / N: 42.4610
4it [14:59, 283.25s/it]
NLL / N: 43.8733
5it [17:37, 245.96s/it]
NLL / N: 44.9294
6it [18:19, 184.48s/it]
NLL / N: 44.9780
7it [20:59, 177.15s/it]
NLL / N: 44.7984
8it [22:34, 152.53s/it]
NLL / N: 44.7538
9it [24:20, 138.62s/it]
NLL / N: 44.8555
10it [26:02, 156.22s/it]
preds: tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 2., 1., 2.,
        0., 2., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 0., 2., 2., 2., 0., 2., 2., 1., 2., 2., 2., 2., 2., 2., 2., 2., 0.,
        2., 2., 2., 1., 2., 2., 2., 3., 2., 2., 2., 3., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 0., 2., 2., 2., 2., 0., 2., 2., 2., 2., 0., 2., 2., 2.,
        1., 2., 2., 2., 2., 0., 2., 2., 2., 2., 2., 2., 2., 0., 2., 3., 2., 2.,
        2., 0., 2., 2.

In [16]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_sin_K5_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 5

/mnt/Cohortney/src/mds20_cohortney-main/src/DMHP/HP.py:26: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  result = lambda w0: integrate.quad(upper_bound, w0, np.inf)
4.48948948948949
D = 15
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 48.8000
1it [17:58, 1078.47s/it]
NLL / N: 43.6074
2it [20:15, 795.93s/it] 
NLL / N: 45.6610
3it [20:47, 566.82s/it]
NLL / N: 44.6195
4it [23:37, 447.82s/it]
NLL / N: 43.9098
5it [26:16, 361.18s/it]
NLL / N: 43.8271
6it [27:41, 278.23s/it]
NLL / N: 43.8653
7it [30:50, 251.66s/it]
NLL / N: 43.8961
8it [34:09, 235.88s/it]
NLL / N: 43.8894
9it [37:05, 217.82s/it]
NLL / N: 43.8397
10it [38:12, 229.26s/it]
preds: tensor([1., 1., 4., 4., 1., 4., 0., 0., 4., 2., 1., 4., 4., 4., 2., 1., 1., 1.,
        2., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 1., 4.,
        4., 4., 1., 4., 1., 4., 1., 1., 1., 4., 4., 1.

0it [00:00, ?it/s]
NLL / N: 49.5569
1it [19:18, 1158.91s/it]
NLL / N: 45.0632
2it [20:01, 823.93s/it] 
NLL / N: 45.5544
3it [22:17, 617.74s/it]
NLL / N: 45.4777
4it [23:36, 455.93s/it]
NLL / N: 45.0294
5it [24:29, 335.04s/it]
NLL / N: 44.5365
6it [25:23, 250.87s/it]
NLL / N: 44.0934
7it [26:26, 194.56s/it]
NLL / N: 43.8539
8it [27:27, 154.52s/it]
NLL / N: 43.5872
9it [28:29, 126.63s/it]
NLL / N: 43.4698
10it [29:35, 177.52s/it]
preds: tensor([0., 3., 4., 2., 3., 2., 1., 4., 2., 4., 3., 2., 2., 1., 4., 3., 0., 0.,
        4., 2., 2., 0., 2., 2., 2., 2., 2., 2., 3., 0., 2., 2., 0., 2., 0., 2.,
        2., 4., 2., 2., 0., 2., 0., 2., 4., 2., 2., 0., 0., 3., 1., 2., 3., 0.,
        2., 2., 2., 2., 2., 0., 2., 0., 2., 0., 3., 3., 0., 2., 2., 0., 0., 1.,
        4., 3., 0., 2., 0., 3., 0., 0., 0., 2., 0., 2., 0., 2., 0., 2., 4., 0.,
        3., 2., 4., 2., 2., 2., 4., 2., 4., 2., 0., 0., 4., 2., 0., 0., 2., 4.,
        2., 2., 3., 3., 2., 0., 0., 0., 4., 0., 2., 4., 3., 2., 0., 0., 0., 2.,
 

In [17]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_trunc_K2_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 2

1.3063063063063063
D = 5
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 40.2789
1it [01:29, 89.51s/it]
NLL / N: 32.7540
2it [01:40, 65.94s/it]
NLL / N: 31.4301
3it [01:58, 51.57s/it]
NLL / N: 32.9155
4it [02:16, 41.37s/it]
NLL / N: 35.0991
5it [02:39, 35.91s/it]
NLL / N: 36.1445
6it [03:03, 32.51s/it]
NLL / N: 36.4685
7it [03:33, 31.72s/it]
NLL / N: 36.5426
8it [04:04, 31.52s/it]
NLL / N: 36.6802
9it [04:33, 30.65s/it]
NLL / N: 36.9503
10it [05:02, 30.27s/it]
preds: tensor([1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
        0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0.,
        1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1.,
        0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
        0., 1., 1., 1., 1., 0., 

In [18]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_trunc_K3_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 3

/mnt/Cohortney/src/mds20_cohortney-main/src/DMHP/HP.py:26: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  result = lambda w0: integrate.quad(upper_bound, w0, np.inf)
2.942942942942943
D = 10
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 30.1863
1it [05:22, 322.34s/it]
NLL / N: 20.0537
2it [05:39, 230.72s/it]
NLL / N: 20.6514
3it [06:04, 169.12s/it]
NLL / N: 24.3713
4it [06:36, 127.82s/it]
NLL / N: 26.4713
5it [07:11, 99.96s/it] 
NLL / N: 27.8994
6it [07:48, 81.23s/it]
NLL / N: 29.4386
7it [08:29, 69.28s/it]
NLL / N: 30.6633
8it [09:11, 60.81s/it]
NLL / N: 31.4210
9it [09:54, 55.46s/it]
NLL / N: 31.8556
10it [10:36, 63.67s/it]
preds: tensor([2., 0., 2., 0., 0., 0., 1., 2., 2., 2., 2., 2., 1., 0., 2., 2., 2., 2.,
        1., 1., 2., 2., 2., 1., 1., 2., 0., 2., 0., 2., 2., 0., 2., 2., 2., 2.,
        2., 0., 2., 0., 0., 1., 0., 1., 0., 0., 2., 2., 2., 

In [19]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_trunc_K4_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 4

3.873873873873874
D = 13
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 23.3802
1it [10:46, 646.60s/it]
NLL / N: 21.1727
2it [11:30, 465.91s/it]
NLL / N: 24.7051
3it [12:21, 341.45s/it]
NLL / N: 29.4924
4it [13:15, 254.93s/it]
NLL / N: 31.6592
5it [14:02, 192.57s/it]
NLL / N: 32.7988
6it [14:59, 151.94s/it]
NLL / N: 33.3581
7it [16:00, 124.74s/it]
NLL / N: 33.5032
8it [16:58, 104.78s/it]
NLL / N: 33.3946
9it [18:02, 92.54s/it] 
NLL / N: 33.2836
10it [19:04, 114.41s/it]
preds: tensor([3., 3., 0., 3., 2., 3., 3., 3., 3., 2., 3., 3., 0., 3., 3., 2., 3., 3.,
        0., 3., 3., 3., 3., 2., 3., 2., 3., 0., 2., 2., 3., 3., 3., 3., 0., 3.,
        0., 0., 3., 3., 3., 2., 3., 0., 3., 3., 2., 2., 3., 0., 3., 2., 3., 3.,
        0., 3., 3., 3., 3., 3., 3., 3., 3., 0., 3., 3., 3., 3., 3., 3., 3., 2.,
        3., 3., 3., 3., 3., 3., 3., 0., 3., 3., 3., 3., 3., 0., 0., 0., 3., 0.,
        3., 3., 0., 3., 2., 3., 3., 3., 3., 3., 3., 0., 3., 2., 3., 3., 0., 2.,
        3., 0., 0., 3.

In [20]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_trunc_K5_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 5

4.504504504504505
D = 15
============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 25.6909
1it [15:24, 924.29s/it]
NLL / N: 23.3075
2it [16:05, 659.34s/it]
NLL / N: 25.1429
3it [17:02, 478.56s/it]
NLL / N: 27.1810
4it [18:12, 356.00s/it]
NLL / N: 30.1141
5it [19:22, 270.27s/it]
NLL / N: 34.7292
6it [20:30, 209.48s/it]
NLL / N: 36.5051
7it [21:49, 170.44s/it]
NLL / N: 36.8267
8it [23:01, 140.83s/it]
NLL / N: 36.9517
9it [23:56, 115.26s/it]
NLL / N: 36.9355
10it [24:53, 149.30s/it]
preds: tensor([4., 3., 1., 1., 0., 1., 0., 2., 0., 0., 0., 0., 0., 1., 0., 4., 2., 0.,
        4., 4., 0., 1., 1., 1., 1., 4., 1., 4., 2., 4., 2., 0., 1., 1., 3., 0.,
        1., 0., 4., 0., 0., 1., 4., 1., 2., 2., 0., 4., 1., 4., 0., 2., 4., 3.,
        1., 4., 0., 0., 4., 0., 0., 1., 2., 4., 2., 1., 3., 2., 2., 2., 1., 0.,
        3., 0., 1., 1., 4., 1., 2., 0., 1., 1., 0., 1., 0., 2., 3., 2., 3., 1.,
        0., 1., 4., 4., 1., 2., 3., 0., 1., 2., 2., 2., 4., 2., 3., 2., 1., 2.,
        0., 0., 4., 2.

0it [00:00, ?it/s]
NLL / N: 27.6696
1it [13:44, 824.44s/it]
NLL / N: 28.7889
2it [14:09, 584.64s/it]
NLL / N: 30.1517
3it [14:41, 418.85s/it]
NLL / N: 30.9774
4it [15:21, 305.06s/it]
NLL / N: 31.4665
5it [16:05, 227.00s/it]
NLL / N: 32.4903
6it [16:55, 173.82s/it]
NLL / N: 34.1370
7it [17:52, 138.73s/it]
NLL / N: 34.8008
8it [18:48, 113.76s/it]
NLL / N: 35.1362
9it [19:43, 96.21s/it] 
NLL / N: 35.2512
10it [20:38, 123.83s/it]
preds: tensor([0., 4., 1., 4., 4., 2., 4., 0., 4., 4., 0., 4., 4., 4., 4., 1., 2., 4.,
        0., 0., 4., 3., 1., 1., 3., 3., 2., 0., 4., 0., 2., 4., 4., 0., 1., 4.,
        0., 4., 0., 4., 0., 4., 0., 1., 2., 0., 0., 4., 1., 0., 0., 2., 4., 4.,
        0., 0., 4., 2., 4., 0., 4., 1., 0., 0., 0., 1., 4., 0., 4., 2., 0., 4.,
        4., 4., 1., 0., 1., 2., 0., 2., 2., 1., 4., 1., 4., 0., 4., 0., 4., 1.,
        4., 1., 1., 0., 2., 0., 3., 4., 0., 2., 4., 0., 0., 2., 3., 0., 2., 0.,
        4., 4., 0., 2., 4., 1., 4., 4., 4., 3., 4., 0., 2., 0., 0., 0., 4., 2.,
   

In [ ]:
slkdjlksdjgl

### Const basis

In [24]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K3_C1' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 3

============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 97.9981
1it [03:14, 194.28s/it]
NLL / N: 4.5288
2it [03:54, 148.13s/it]
NLL / N: 5.5526
3it [04:39, 117.25s/it]
NLL / N: 4.6715
4it [05:42, 100.99s/it]
NLL / N: 5.1024
5it [06:58, 93.35s/it] 
NLL / N: 4.7893
6it [08:32, 93.63s/it]
NLL / N: 4.4662
7it [10:11, 95.01s/it]
NLL / N: 4.4574
8it [11:45, 94.71s/it]
NLL / N: 4.9491
9it [13:04, 90.12s/it]
NLL / N: 4.6537
10it [14:26, 86.68s/it]
preds: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [28]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K2_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 3

============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 102.7772
1it [00:25, 25.45s/it]
NLL / N: 94.5940
2it [00:33, 20.12s/it]
NLL / N: 92.3320
3it [00:41, 16.72s/it]
NLL / N: 99.6658
4it [00:51, 14.43s/it]
NLL / N: 99.5787
5it [01:05, 14.42s/it]
NLL / N: 97.4586
6it [01:19, 14.37s/it]
NLL / N: 96.5282
7it [01:35, 14.74s/it]
NLL / N: 97.1572
8it [01:51, 15.12s/it]
NLL / N: 97.6342
9it [02:06, 15.05s/it]
NLL / N: 97.8500
10it [02:22, 14.21s/it]
preds: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 

2it [00:34, 20.95s/it]
NLL / N: 94.0954
3it [00:43, 17.48s/it]
NLL / N: 104.7397
4it [00:52, 15.01s/it]
NLL / N: 103.6284
5it [01:05, 14.35s/it]
NLL / N: 100.6346
6it [01:20, 14.42s/it]
NLL / N: 99.1292
7it [01:35, 14.67s/it]
NLL / N: 99.1130
8it [01:51, 14.96s/it]
NLL / N: 99.2896
9it [02:05, 14.93s/it]
NLL / N: 99.3922
10it [02:22, 14.29s/it]
preds: tensor([2., 2., 1., 2., 1., 2., 1., 1., 2., 1., 2., 2., 2., 2., 2., 2., 1., 2.,
        2., 2., 2., 2., 2., 1., 1., 2., 2., 1., 1., 2., 2., 1., 2., 2., 1., 1.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 2., 1., 1., 1., 1., 2.,
        2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 2., 2., 1., 1.,
        2., 2., 1., 2., 2., 1., 2., 1., 1., 2., 2., 2., 2., 2., 2., 2., 1., 2.,
        2., 2., 1., 2., 1., 2., 2., 2., 1., 1., 2., 2., 2., 2., 2., 2., 2., 1.,
        2., 2., 2., 2., 2., 2., 2., 1., 1., 2., 2., 2., 2., 1., 1., 1., 2., 2.,
        2., 2., 2., 2., 1., 1., 1., 2., 2., 1., 1., 2., 1., 2., 1., 2., 2., 2.,
      

5it [01:04, 14.16s/it]
NLL / N: 97.2134
6it [01:18, 13.87s/it]
NLL / N: 98.1340
7it [01:33, 14.47s/it]
NLL / N: 98.0491
8it [01:49, 14.75s/it]
NLL / N: 98.2620
9it [02:06, 15.42s/it]
NLL / N: 98.3074
10it [02:22, 14.26s/it]
preds: tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2

In [25]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K3_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 3

============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 117.9799
1it [00:36, 36.67s/it]
NLL / N: 116.4016
2it [00:47, 29.03s/it]
NLL / N: 116.5795
3it [01:00, 24.04s/it]
NLL / N: 117.1189
4it [01:17, 21.91s/it]
NLL / N: 114.8261
5it [01:36, 21.27s/it]
NLL / N: 113.6377
6it [02:04, 23.22s/it]
NLL / N: 113.5000
7it [02:31, 24.34s/it]
NLL / N: 113.4822
8it [02:57, 24.73s/it]
NLL / N: 113.3760
9it [03:22, 24.83s/it]
NLL / N: 113.2301
10it [03:48, 22.85s/it]
preds: tensor([2., 0., 0., 2., 0., 2., 0., 0., 2., 0., 2., 2., 2., 2., 2., 2., 0., 0.,
        2., 2., 2., 0., 2., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 2., 0., 0.,
        2., 0., 2., 2., 0., 2., 2., 0., 0., 2., 0., 0., 2., 0., 0., 0., 0., 2.,
        2., 2., 2., 2., 2., 0., 0., 0., 2., 0., 0., 0., 0., 2., 0., 2., 0., 0.,
        0., 2., 0., 2., 0., 0., 2., 0., 0., 2., 2., 2., 2., 2., 0., 2., 0., 2.,
        2., 2., 0., 0., 0., 0., 2., 2., 0., 0., 2., 2., 2., 0., 2., 2., 2., 0.,
        2., 2., 2., 2., 2., 2., 2., 0., 0., 0.,

0it [00:00, ?it/s]
NLL / N: 116.9857
1it [00:35, 35.73s/it]
NLL / N: 121.2273
2it [00:45, 28.05s/it]
NLL / N: 117.2387
3it [01:00, 24.06s/it]
NLL / N: 118.1590
4it [01:19, 22.49s/it]
NLL / N: 117.0262
5it [01:41, 22.45s/it]
NLL / N: 115.0162
6it [02:04, 22.53s/it]
NLL / N: 113.5995
7it [02:29, 23.22s/it]
NLL / N: 113.0739
8it [02:53, 23.49s/it]
NLL / N: 112.9087
9it [03:18, 24.06s/it]
NLL / N: 112.7584
10it [03:43, 22.37s/it]
preds: tensor([1., 1., 1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
        2., 2., 2., 1., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1.,
        2., 1., 2., 2., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 2.,
        1., 2., 1., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
        1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 2., 2., 1., 2., 1., 1., 1., 2.,
        2., 1., 1., 1., 1., 1., 2., 2., 1., 1., 1., 1., 2., 1., 2., 2., 2., 1.,
        2., 2., 1., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
   

0it [00:00, ?it/s]
NLL / N: 117.1472
1it [00:36, 36.69s/it]
NLL / N: 108.9377
2it [00:54, 30.93s/it]
NLL / N: 108.6473
3it [01:08, 26.07s/it]
NLL / N: 110.0766
4it [01:24, 22.99s/it]
NLL / N: 116.5888
5it [01:45, 22.28s/it]
NLL / N: 115.5877
6it [02:07, 22.31s/it]
NLL / N: 113.4622
7it [02:33, 23.24s/it]
NLL / N: 112.6861
8it [02:57, 23.44s/it]
NLL / N: 112.5842
9it [03:21, 23.65s/it]
NLL / N: 112.5469
10it [03:46, 22.60s/it]
preds: tensor([1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
        2., 2., 2., 1., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1.,
        2., 1., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
        1., 2., 1., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
        1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1.,
        2., 1., 1., 1., 1., 1., 2., 2., 1., 1., 1., 1., 2., 1., 2., 2., 2., 1.,
        2., 2., 1., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
   

In [26]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K4_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 4

============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 115.3922
1it [00:58, 58.57s/it]
NLL / N: 111.6379
2it [01:14, 45.85s/it]
NLL / N: 111.5005
3it [01:33, 37.81s/it]
NLL / N: 111.6290
4it [01:59, 34.26s/it]
NLL / N: 111.8191
5it [02:28, 32.75s/it]
NLL / N: 111.8693
6it [03:06, 34.03s/it]
NLL / N: 111.8745
7it [03:40, 34.25s/it]
NLL / N: 111.8756
8it [04:16, 34.75s/it]
NLL / N: 111.8818
9it [04:51, 34.84s/it]
NLL / N: 111.8949
10it [05:27, 32.76s/it]
preds: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

0it [00:00, ?it/s]
NLL / N: 113.9042
1it [00:49, 49.36s/it]
NLL / N: 112.8532
2it [01:05, 39.26s/it]
NLL / N: 113.2953
3it [01:24, 33.39s/it]
NLL / N: 112.6256
4it [01:51, 31.29s/it]
NLL / N: 112.6881
5it [02:18, 30.23s/it]
NLL / N: 112.6289
6it [02:48, 30.08s/it]
NLL / N: 112.6112
7it [03:23, 31.58s/it]
NLL / N: 112.4773
8it [03:59, 32.85s/it]
NLL / N: 113.1728
9it [04:33, 33.28s/it]
NLL / N: 113.3247
10it [05:12, 31.25s/it]
preds: tensor([1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0.,
        0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
   

0it [00:00, ?it/s]
NLL / N: 114.6417
1it [00:49, 49.73s/it]
NLL / N: 111.1216
2it [01:04, 39.34s/it]
NLL / N: 111.6099
3it [01:24, 33.49s/it]
NLL / N: 111.9083
4it [01:53, 32.00s/it]
NLL / N: 112.3132
5it [02:22, 31.33s/it]
NLL / N: 113.0500
6it [02:53, 30.98s/it]
NLL / N: 113.8409
7it [03:28, 32.35s/it]
NLL / N: 113.8857
8it [04:03, 32.98s/it]
NLL / N: 112.8151
9it [04:37, 33.26s/it]
NLL / N: 112.9450
10it [05:15, 31.58s/it]
preds: tensor([0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 3., 3., 0., 0., 0.,
        0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 3., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   

In [27]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/K5_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 5

============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 116.7175
1it [01:12, 72.64s/it]
NLL / N: 107.9537
2it [01:33, 57.04s/it]
NLL / N: 108.7138
3it [01:56, 46.83s/it]
NLL / N: 112.2975
4it [02:27, 42.13s/it]
NLL / N: 111.3844
5it [03:04, 40.57s/it]
NLL / N: 109.7413
6it [03:44, 40.33s/it]
NLL / N: 108.7369
7it [04:26, 41.05s/it]
NLL / N: 108.5636
8it [05:10, 41.77s/it]
NLL / N: 108.5314
9it [05:54, 42.34s/it]
NLL / N: 108.5219
10it [06:36, 39.61s/it]
preds: tensor([4., 4., 4., 1., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 1., 4., 4.,
        1., 1., 1., 4., 1., 4., 4., 4., 4., 4., 4., 1., 4., 4., 4., 4., 3., 4.,
        4., 4., 1., 1., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 1.,
        4., 1., 4., 1., 1., 4., 4., 4., 4., 4., 4., 4., 4., 1., 4., 4., 4., 4.,
        4., 1., 4., 4., 4., 4., 4., 4., 4., 4., 1., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4., 4., 4., 1., 1., 4., 4., 4., 4., 1., 4., 4., 1., 1., 4.,
        1., 4., 4., 1., 1., 1., 4., 4., 4., 4.,

0it [00:00, ?it/s]
NLL / N: 116.0418
1it [01:16, 76.80s/it]
NLL / N: 107.3645
2it [01:35, 59.33s/it]
NLL / N: 109.2817
3it [02:01, 49.26s/it]
NLL / N: 110.5163
4it [02:31, 43.62s/it]
NLL / N: 111.1006
5it [03:07, 41.17s/it]
NLL / N: 111.1964
6it [03:45, 40.28s/it]
NLL / N: 111.6605
7it [04:29, 41.53s/it]
NLL / N: 111.4248
8it [05:11, 41.73s/it]
NLL / N: 110.8024
9it [05:54, 42.05s/it]
NLL / N: 110.4607
10it [06:40, 40.01s/it]
preds: tensor([2., 1., 1., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 1., 1.,
        3., 3., 3., 1., 3., 1., 1., 1., 1., 1., 1., 3., 1., 1., 1., 1., 2., 1.,
        3., 1., 3., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 3.,
        1., 3., 1., 3., 3., 1., 1., 1., 1., 1., 1., 1., 1., 3., 1., 1., 2., 1.,
        1., 3., 1., 1., 1., 1., 3., 2., 1., 1., 3., 1., 1., 1., 1., 1., 1., 3.,
        3., 1., 1., 1., 1., 1., 3., 3., 1., 1., 1., 1., 3., 1., 1., 3., 3., 1.,
        3., 3., 1., 3., 3., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 3.,
   

0it [00:00, ?it/s]
NLL / N: 115.3725
1it [01:13, 73.56s/it]
NLL / N: 106.1763
2it [01:32, 57.05s/it]
NLL / N: 109.6124
3it [01:56, 47.25s/it]
NLL / N: 108.6687
4it [02:27, 42.40s/it]
NLL / N: 107.8879
5it [03:04, 40.90s/it]
NLL / N: 113.1172
6it [03:43, 40.24s/it]
NLL / N: 112.3345
7it [04:25, 40.77s/it]
NLL / N: 110.5630
8it [05:07, 41.06s/it]
NLL / N: 109.5466
9it [05:50, 41.58s/it]
NLL / N: 109.2627
10it [06:32, 39.30s/it]
preds: tensor([0., 0., 4., 0., 4., 4., 4., 4., 0., 4., 0., 0., 0., 0., 0., 0., 4., 0.,
        0., 3., 0., 4., 0., 4., 4., 4., 0., 4., 4., 0., 0., 4., 4., 0., 4., 4.,
        0., 0., 0., 0., 0., 0., 0., 4., 4., 0., 4., 4., 4., 4., 4., 4., 4., 3.,
        0., 0., 0., 0., 0., 0., 4., 4., 0., 4., 4., 4., 4., 0., 4., 0., 0., 4.,
        4., 0., 4., 0., 4., 4., 0., 4., 4., 4., 0., 0., 0., 0., 0., 0., 4., 0.,
        0., 0., 4., 4., 4., 0., 0., 0., 4., 4., 0., 0., 0., 0., 0., 0., 0., 4.,
        0., 0., 0., 0., 0., 0., 0., 4., 4., 4., 4., 0., 4., 4., 4., 0., 4., 0.,
   

0it [00:00, ?it/s]
NLL / N: 110.7695
1it [01:12, 72.90s/it]
NLL / N: 106.2856
2it [01:34, 57.55s/it]
NLL / N: 107.4120
3it [01:58, 47.47s/it]
NLL / N: 109.5979
4it [02:26, 41.59s/it]
NLL / N: 111.0118
5it [03:01, 39.56s/it]
NLL / N: 110.5673
6it [03:42, 40.07s/it]
NLL / N: 110.2845
7it [04:26, 41.35s/it]
NLL / N: 109.9783
8it [05:11, 42.19s/it]
NLL / N: 109.6809
9it [05:53, 42.41s/it]
NLL / N: 109.7115
10it [06:40, 40.00s/it]
preds: tensor([2., 2., 2., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2.,
        1., 1., 1., 2., 1., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 2.,
        1., 2., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1.,
        2., 1., 2., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 2.,
        2., 1., 2., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 2., 2., 2., 2., 1.,
        1., 2., 2., 2., 2., 2., 1., 1., 2., 2., 2., 2., 1., 2., 2., 1., 1., 2.,
        1., 1., 2., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1.,
   

0it [00:00, ?it/s]
NLL / N: 114.6704
1it [01:15, 75.19s/it]
NLL / N: 111.2324
2it [01:33, 58.18s/it]
NLL / N: 113.5559
3it [01:58, 48.17s/it]
NLL / N: 112.0714
4it [02:29, 42.97s/it]
NLL / N: 110.8844
5it [03:07, 41.68s/it]
NLL / N: 110.5865
6it [03:48, 41.43s/it]
NLL / N: 110.1704
7it [04:31, 41.85s/it]
NLL / N: 110.1404
8it [05:18, 43.20s/it]
NLL / N: 110.1696
9it [06:01, 43.39s/it]
NLL / N: 110.1840
10it [06:42, 40.22s/it]
preds: tensor([3., 4., 4., 0., 4., 4., 4., 4., 4., 3., 4., 4., 4., 4., 4., 0., 4., 4.,
        0., 0., 0., 4., 0., 4., 4., 4., 4., 4., 4., 0., 4., 4., 4., 4., 3., 4.,
        0., 4., 0., 0., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 3., 0.,
        4., 0., 4., 0., 0., 4., 4., 4., 4., 4., 4., 4., 4., 0., 4., 4., 3., 4.,
        4., 0., 4., 4., 4., 4., 0., 4., 4., 4., 0., 4., 4., 4., 4., 4., 4., 0.,
        0., 4., 4., 4., 4., 4., 0., 0., 4., 4., 4., 4., 0., 4., 4., 0., 0., 4.,
        0., 0., 4., 0., 0., 0., 0., 4., 4., 4., 4., 4., 4., 4., 4., 3., 4., 0.,
   

Consistency: 0.6869

Purity: 0.6126+-0.0396


### Sin

In [5]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_sin_K2_C5' \
    --verbose \
    --nruns 5 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 2

============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 83.4989
1it [00:56, 56.97s/it]
NLL / N: 48.2714
2it [01:09, 43.60s/it]
NLL / N: 41.9987
3it [01:24, 35.19s/it]
NLL / N: 42.2544
4it [01:44, 30.39s/it]
NLL / N: 44.1367
5it [02:07, 28.34s/it]
NLL / N: 44.6565
6it [02:34, 27.90s/it]
NLL / N: 44.8498
7it [03:03, 28.28s/it]
NLL / N: 44.9087
8it [03:31, 28.23s/it]
NLL / N: 44.8965
9it [03:59, 28.08s/it]
NLL / N: 44.8736
10it [04:28, 26.89s/it]
preds: tensor([1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
        1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
        0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1.,
        0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0

5it [02:01, 26.97s/it]
NLL / N: 44.9050
6it [02:28, 27.00s/it]
NLL / N: 44.8878
7it [02:56, 27.37s/it]
NLL / N: 44.8666
8it [03:24, 27.33s/it]
NLL / N: 44.8496
9it [03:52, 27.55s/it]
NLL / N: 44.8378
10it [04:20, 26.03s/it]
preds: tensor([0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
        0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1.,
        1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
        1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1.,
        1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1

In [6]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_sin_K3_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 3

============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 79.5502
1it [01:25, 85.35s/it]
NLL / N: 55.2149
2it [01:42, 64.89s/it]
NLL / N: 49.9521
3it [02:02, 51.56s/it]
NLL / N: 51.1540
4it [02:27, 43.43s/it]
NLL / N: 51.5032
5it [02:57, 39.46s/it]
NLL / N: 51.5688
6it [03:34, 38.74s/it]
NLL / N: 52.0782
7it [04:13, 38.86s/it]
NLL / N: 52.4183
8it [04:50, 38.17s/it]
NLL / N: 52.5446
9it [05:30, 38.72s/it]
NLL / N: 52.5685
10it [06:07, 36.74s/it]
preds: tensor([1., 0., 1., 2., 2., 2., 2., 2., 0., 2., 1., 2., 2., 2., 1., 1., 2., 2.,
        0., 1., 1., 1., 2., 1., 2., 1., 2., 1., 2., 2., 0., 2., 2., 1., 2., 0.,
        1., 1., 1., 0., 2., 2., 1., 2., 2., 2., 0., 1., 1., 1., 2., 2., 2., 2.,
        2., 2., 1., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1., 2., 2., 2., 2., 0.,
        2., 2., 2., 2., 1., 0., 1., 2., 1., 1., 1., 1., 2., 0., 2., 1., 2., 2.,
        1., 1., 2., 2., 1., 2., 0., 1., 0., 1., 1., 1., 1., 1., 2., 1., 0., 1.,
        1., 2., 2., 1., 2., 2., 1., 2., 2., 1., 2., 2., 1

In [7]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_sin_K4_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 4

============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 80.7010
1it [01:57, 117.82s/it]
NLL / N: 53.5047
2it [02:17, 88.34s/it] 
NLL / N: 50.7064
3it [02:40, 68.72s/it]
NLL / N: 51.3624
4it [03:09, 56.84s/it]
NLL / N: 52.3805
5it [03:43, 49.91s/it]
NLL / N: 52.5952
6it [04:24, 47.26s/it]
NLL / N: 52.6554
7it [05:08, 46.32s/it]
NLL / N: 52.6868
8it [05:52, 45.74s/it]
NLL / N: 52.7109
9it [06:37, 45.55s/it]
NLL / N: 52.7277
10it [07:22, 44.28s/it]
preds: tensor([3., 1., 1., 3., 1., 3., 1., 3., 1., 1., 3., 0., 1., 3., 1., 1., 0., 1.,
        3., 1., 1., 1., 1., 1., 1., 1., 1., 3., 1., 1., 1., 3., 1., 1., 1., 3.,
        1., 3., 1., 1., 1., 2., 1., 3., 2., 1., 3., 1., 1., 1., 1., 1., 3., 1.,
        1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 3., 1., 1.,
        1., 3., 1., 1., 1., 1., 3., 3., 3., 3., 1., 1., 1., 3., 2., 1., 3., 3.,
        0., 1., 1., 1., 1., 1., 1., 3., 1., 3., 3., 3., 1., 3., 1., 1., 1., 1.,
        1., 2., 1., 1., 1., 1., 3., 1., 3., 0., 3., 1.,

In [8]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_sin_K5_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 5

============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 84.5146
1it [02:07, 127.38s/it]
NLL / N: 55.0438
2it [02:29, 95.90s/it] 
NLL / N: 49.4957
3it [02:57, 75.31s/it]
NLL / N: 50.7312
4it [03:29, 62.48s/it]
NLL / N: 51.4063
5it [04:09, 55.68s/it]
NLL / N: 51.5719
6it [04:54, 52.44s/it]
NLL / N: 51.6481
7it [05:43, 51.59s/it]
NLL / N: 51.6790
8it [06:31, 50.48s/it]
NLL / N: 51.6835
9it [07:20, 49.89s/it]
NLL / N: 51.6781
10it [08:12, 49.25s/it]
preds: tensor([0., 1., 4., 2., 3., 0., 1., 4., 3., 4., 1., 3., 3., 1., 4., 3., 3., 0.,
        1., 2., 3., 3., 2., 2., 3., 2., 1., 3., 3., 3., 3., 2., 4., 4., 3., 3.,
        3., 3., 3., 3., 3., 1., 3., 0., 4., 2., 2., 3., 3., 1., 1., 4., 1., 3.,
        2., 2., 2., 0., 3., 2., 0., 1., 3., 1., 2., 3., 3., 3., 3., 1., 2., 1.,
        4., 3., 2., 4., 3., 1., 1., 1., 1., 2., 3., 3., 3., 2., 3., 2., 4., 3.,
        2., 3., 4., 3., 0., 2., 4., 2., 4., 0., 3., 1., 4., 3., 1., 3., 2., 1.,
        0., 3., 1., 1., 2., 3., 4., 3., 4., 3., 0., 4.,

0it [00:00, ?it/s]
NLL / N: 81.0565
1it [02:07, 127.86s/it]
NLL / N: 53.9152
2it [02:30, 96.39s/it] 
NLL / N: 50.0740
3it [02:57, 75.55s/it]
NLL / N: 51.1051
4it [03:32, 63.20s/it]
NLL / N: 51.5308
5it [04:12, 56.25s/it]
NLL / N: 51.6365
6it [04:59, 53.54s/it]
NLL / N: 51.7178
7it [05:48, 52.36s/it]
NLL / N: 51.7319
8it [06:37, 51.27s/it]
NLL / N: 51.7186
9it [07:27, 50.79s/it]
NLL / N: 51.7006
10it [08:16, 49.64s/it]
preds: tensor([1., 0., 4., 2., 0., 1., 0., 3., 4., 4., 0., 0., 0., 3., 3., 0., 0., 1.,
        3., 2., 0., 4., 2., 2., 2., 2., 0., 0., 0., 2., 0., 2., 4., 4., 2., 0.,
        0., 4., 2., 4., 0., 0., 0., 1., 4., 2., 2., 0., 0., 0., 3., 4., 0., 2.,
        2., 2., 2., 1., 0., 2., 1., 0., 2., 4., 2., 2., 2., 2., 0., 0., 2., 3.,
        4., 2., 2., 4., 2., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 4., 0.,
        2., 0., 4., 0., 1., 2., 3., 2., 4., 1., 0., 0., 3., 0., 4., 0., 2., 0.,
        1., 0., 0., 0., 2., 4., 4., 4., 4., 2., 1., 3., 3., 0., 2., 0., 0., 0.,
        2.,

### Trunc

In [9]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_trunc_K2_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 2

============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 65.6834
1it [01:14, 74.05s/it]
NLL / N: 55.0652
2it [01:25, 55.18s/it]
NLL / N: 48.5473
3it [01:39, 42.84s/it]
NLL / N: 47.2479
4it [01:55, 34.92s/it]
NLL / N: 47.7539
5it [02:14, 30.04s/it]
NLL / N: 48.2851
6it [02:36, 27.53s/it]
NLL / N: 48.8032
7it [03:00, 26.69s/it]
NLL / N: 49.6307
8it [03:25, 26.19s/it]
NLL / N: 50.4666
9it [03:51, 25.92s/it]
NLL / N: 50.7423
10it [04:15, 25.58s/it]
preds: tensor([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1.,
        0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
        0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
        1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
        1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1

In [10]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_trunc_K3_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 3

============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 64.8366
1it [01:57, 117.79s/it]
NLL / N: 56.9432
2it [02:12, 86.93s/it] 
NLL / N: 49.4003
3it [02:30, 66.16s/it]
NLL / N: 51.7397
4it [02:52, 52.88s/it]
NLL / N: 53.7824
5it [03:17, 44.68s/it]
NLL / N: 54.8265
6it [03:47, 40.28s/it]
NLL / N: 55.3082
7it [04:22, 38.57s/it]
NLL / N: 55.5023
8it [04:56, 37.09s/it]
NLL / N: 55.5968
9it [05:26, 35.09s/it]
NLL / N: 55.6453
10it [05:58, 35.90s/it]
preds: tensor([2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 1., 1., 1., 2., 2., 1.,
        2., 1., 2., 1., 2., 1., 1., 1., 1., 1., 2., 2., 1., 1., 1., 1., 1., 1.,
        1., 1., 2., 1., 1., 1., 0., 1., 2., 1., 1., 2., 1., 1., 0., 1., 2., 2.,
        2., 1., 1., 2., 2., 1., 2., 1., 1., 1., 1., 2., 1., 2., 2., 2., 2., 1.,
        1., 2., 2., 2., 2., 1., 1., 2., 1., 1., 2., 2., 1., 2., 2., 1., 1., 2.,
        0., 1., 1., 1., 1., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
        1., 2., 1., 1., 1., 1., 1., 2., 1., 2., 1., 1.,

In [11]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_trunc_K4_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 4

============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 67.5650
1it [02:26, 146.45s/it]
NLL / N: 58.0948
2it [02:46, 108.40s/it]
NLL / N: 47.7330
3it [03:11, 83.41s/it] 
NLL / N: 45.9878
4it [03:40, 67.33s/it]
NLL / N: 48.3206
5it [04:16, 57.88s/it]
NLL / N: 49.7780
6it [04:59, 53.21s/it]
NLL / N: 51.0269
7it [05:48, 51.91s/it]
NLL / N: 51.3667
8it [06:35, 50.61s/it]
NLL / N: 51.4222
9it [07:21, 49.33s/it]
NLL / N: 51.4324
10it [08:07, 48.74s/it]
preds: tensor([3., 3., 2., 0., 0., 0., 1., 3., 2., 3., 2., 0., 0., 2., 0., 3., 0., 3.,
        3., 0., 0., 3., 2., 0., 1., 3., 2., 2., 0., 0., 1., 2., 2., 0., 0., 0.,
        3., 1., 0., 3., 3., 1., 3., 2., 0., 1., 3., 0., 0., 3., 3., 1., 3., 0.,
        3., 0., 3., 2., 0., 0., 3., 1., 3., 3., 2., 2., 3., 2., 3., 2., 2., 0.,
        0., 3., 3., 3., 0., 3., 0., 3., 3., 3., 2., 2., 0., 0., 0., 1., 1., 1.,
        0., 0., 3., 2., 3., 3., 3., 3., 1., 0., 2., 2., 3., 0., 3., 2., 2., 3.,
        0., 3., 3., 0., 0., 2., 0., 2., 3., 3., 3., 3.

In [12]:
!$sys.executable -m src.DMHP.main \
    --data_dir 'data/simulated_Hawkes/tmp_trunc_K5_C5' \
    --verbose \
    --nruns 2 \
    --not_datetime \
    --ext csv \
    --nmb_cluster 5

============= RUN 1 ===============
0it [00:00, ?it/s]
NLL / N: 70.2492
1it [02:55, 175.36s/it]
NLL / N: 58.4964
2it [03:17, 129.25s/it]
NLL / N: 47.5839
3it [03:46, 99.23s/it] 
NLL / N: 45.9253
4it [04:19, 79.56s/it]
NLL / N: 48.7099
5it [05:00, 67.80s/it]
NLL / N: 49.4237
6it [05:46, 61.40s/it]
NLL / N: 49.8695
7it [06:44, 60.25s/it]
NLL / N: 50.0778
8it [07:35, 57.68s/it]
NLL / N: 50.1492
9it [08:29, 56.39s/it]
NLL / N: 50.1887
10it [09:21, 56.16s/it]
preds: tensor([2., 0., 4., 1., 1., 1., 3., 2., 3., 4., 0., 1., 1., 3., 0., 1., 2., 0.,
        1., 1., 1., 3., 2., 2., 1., 0., 4., 1., 3., 0., 3., 4., 3., 1., 1., 2.,
        0., 1., 2., 1., 1., 1., 1., 2., 3., 0., 4., 3., 1., 1., 1., 1., 3., 3.,
        0., 1., 2., 1., 3., 2., 1., 1., 2., 4., 0., 1., 3., 2., 3., 2., 2., 0.,
        1., 3., 2., 2., 0., 2., 1., 0., 0., 4., 2., 0., 1., 2., 3., 0., 1., 4.,
        1., 4., 1., 2., 2., 2., 0., 0., 1., 2., 1., 1., 2., 3., 1., 1., 1., 2.,
        4., 4., 3., 2., 3., 2., 3., 3., 0., 3., 3., 4.

0it [00:00, ?it/s]
NLL / N: 68.6380
1it [02:51, 171.67s/it]
NLL / N: 58.5253
2it [03:12, 126.55s/it]
NLL / N: 46.2546
3it [03:40, 96.72s/it] 
NLL / N: 47.4597
4it [04:13, 77.64s/it]
NLL / N: 49.1255
5it [04:54, 66.78s/it]
NLL / N: 49.8686
6it [05:41, 60.94s/it]
NLL / N: 50.2192
7it [06:36, 59.09s/it]
NLL / N: 50.3427
8it [07:29, 57.22s/it]
NLL / N: 50.3881
9it [08:22, 56.02s/it]
NLL / N: 50.4125
10it [09:14, 55.45s/it]
preds: tensor([0., 3., 1., 1., 1., 1., 4., 4., 0., 1., 3., 1., 1., 4., 3., 0., 1., 3.,
        1., 0., 1., 4., 0., 0., 1., 3., 1., 1., 4., 3., 4., 1., 0., 0., 1., 0.,
        3., 1., 0., 1., 1., 1., 1., 0., 3., 3., 2., 3., 1., 1., 1., 1., 4., 4.,
        3., 0., 0., 1., 0., 4., 1., 1., 0., 2., 3., 0., 4., 3., 4., 1., 0., 3.,
        1., 4., 3., 0., 3., 4., 1., 4., 3., 2., 0., 3., 1., 4., 4., 3., 0., 2.,
        0., 1., 0., 0., 1., 4., 3., 3., 1., 1., 1., 1., 0., 4., 1., 1., 1., 0.,
        2., 2., 0., 4., 0., 0., 4., 3., 3., 0., 4., 2., 1., 1., 0., 2., 1., 1.,
        2.